In [1]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql import functions as f
import pandas as pd 
import numpy as np
import os 
import xmltodict
import json 
import pandas as pd

In [2]:
spark = SparkSession.builder.appName( 'Ealing' ).getOrCreate()

# Coverting XML to Dict

In [3]:
with open('D:/Side Project/Datasets/Food Health Rating/Data/Ealing.xml') as fd:
    doc = xmltodict.parse(fd.read())

In [4]:
with open('D:/Side Project/Datasets/Food Health Rating/Data/Ealing_Json.txt', 'w') as outfile:
    json.dump(doc, outfile)

In [ ]:
{"FHRSID": "610257",
 "LocalAuthorityBusinessID": "PI/000173335",
 "BusinessName": "97p Stores",
 "BusinessType": "Retailers - other", 
 "BusinessTypeID": "4613",
 "AddressLine1": "66 Broadway", 
 "AddressLine2": "West Ealing",
 "PostCode": "W13 0SY", 
 "RatingValue": "1", 
 "RatingKey": "fhrs_1_en-GB", 
 "RatingDate": "2016-01-30",
 "LocalAuthorityCode": "509",
 "LocalAuthorityName": "Ealing",
 "LocalAuthorityWebSite": "https://www.ealing.gov.uk/info/201150/food_hygiene_and_safety",
 "LocalAuthorityEmailAddress": "foodsafety@ealing.gov.uk",
 "Scores": {"Hygiene": "10", "Structural": "5", "ConfidenceInManagement": "20"},
 "SchemeType": "FHRS", 
 "NewRatingPending": "False", 
 "Geocode": {"Longitude": "-0.32524100000000", "Latitude": "51.51054700000000"}},

In [94]:
ealing_df = pd.DataFrame.from_dict(doc['FHRSEstablishment']['EstablishmentCollection']['EstablishmentDetail'])

In [95]:
def convert_dict_scores( df, scores ):
    
    if scores == 'Y':  #extracting values for Scores 
    
        #create a temporary dataframe to store the dictionary values 

        temp_df = pd.DataFrame( columns = ['Hygiene','Structural','ConfidenceInManagement'] )

        # Populate temp_df dataframe

        for (i,r) in df.iterrows():

            e = r['Scores']

            try:

                temp_df.loc[i] = [e['Hygiene'], e['Structural'], e['ConfidenceInManagement']]

            except TypeError:

                temp_df.loc[i] = 'None'

        df_1 = pd.concat([df, temp_df] , axis = 1)

        del df_1['Scores']
        
    else: #extracting values for geocode
        
        temp_df = pd.DataFrame( columns = ['Longitude','Latitude'] )
    
    # Populate temp_df dataframe
    
        for (i,r) in df.iterrows():

            e = r['Geocode']

            try:

                temp_df.loc[i] = [e['Longitude'], e['Latitude']]

            except TypeError:

                temp_df.loc[i] = 'None'

        df_1 = pd.concat([df, temp_df] , axis = 1)

        del df_1['Geocode']
        
    
    return df_1


In [96]:
ealing_df = convert_dict_scores( ealing_df , 'Y')

In [97]:
ealing_df = convert_dict_scores( ealing_df , 'N')

In [98]:
ealing_df.columns

Index(['FHRSID', 'LocalAuthorityBusinessID', 'BusinessName', 'BusinessType',
       'BusinessTypeID', 'RatingValue', 'RatingKey', 'RatingDate',
       'LocalAuthorityCode', 'LocalAuthorityName', 'LocalAuthorityWebSite',
       'LocalAuthorityEmailAddress', 'SchemeType', 'NewRatingPending',
       'AddressLine1', 'AddressLine2', 'AddressLine3', 'AddressLine4',
       'PostCode', 'RightToReply', 'Hygiene', 'Structural',
       'ConfidenceInManagement', 'Longitude', 'Latitude'],
      dtype='object')

In [102]:
ealing_df = ealing_df.replace(to_replace = 'None' , value = np.nan)

In [104]:
ealing_df

,FHRSID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,RatingValue,RatingKey,RatingDate,LocalAuthorityCode,LocalAuthorityName,...,AddressLine2,AddressLine3,AddressLine4,PostCode,RightToReply,Hygiene,Structural,ConfidenceInManagement,Longitude,Latitude
0,1017491,CH00130,106 Coffee,Manufacturers/packers,7839,AwaitingInspection,fhrs_awaitinginspection_en-GB,{'@xsi:nil': 'true'},509,Ealing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,962896,CP00454,1070 Kitchen,Manufacturers/packers,7839,5,fhrs_5_en-GB,2017-06-14,509,Ealing,...,8 Gorst Road,Park Royal,London,NW10 6LE,NaN,5,5,5,-0.26322300000000,51.52740500000000
2,840152,PI/000182063,11 Coffee & Co,Restaurant/Cafe/Canteen,1,5,fhrs_5_en-GB,2016-01-20,509,Ealing,...,Hanger Lane,Ealing,NaN,W5 3HU,NaN,5,5,5,-0.29072000000000,51.51686500000000
3,590655,PI/000173183,5 Star Superstore,Retailers - other,4613,4,fhrs_4_en-GB,2015-07-14,509,Ealing,...,Greenford,NaN,NaN,UB6 9AT,NaN,10,0,5,-0.35438900000000,51.51866600000000
4,610257,PI/000173335,97p Stores,Retailers - other,4613,1,fhrs_1_en-GB,2016-01-30,509,Ealing,...,West Ealing,NaN,NaN,W13 0SY,NaN,10,5,20,-0.32524100000000,51.51054700000000
5,391740,PI/000054112,A - Z Premier Food And Wine,Retailers - other,4613,5,fhrs_5_en-GB,2016-02-17,509,Ealing,...,Greenford,NaN,NaN,UB6 0ES,NaN,0,0,0,-0.34366700000000,51.55002400000000
6,833588,PI/000061231,A & C Roll Factory,Manufacturers/packers,7839,5,fhrs_5_en-GB,2016-03-16,509,Ealing,...,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN
7,393584,PI/000168095,A 2 Zee's Cafe,Restaurant/Cafe/Canteen,1,2,fhrs_2_en-GB,2017-01-23,509,Ealing,...,West Ealing,London,NaN,W13 0LJ,NaN,10,15,10,-0.32046700000000,51.51377000000000
8,1005361,CP00781,A Demain,Manufacturers/packers,7839,AwaitingInspection,fhrs_awaitinginspection_en-GB,{'@xsi:nil': 'true'},509,Ealing,...,2 Portal Way,Acton,London,W3 6RT,NaN,NaN,NaN,NaN,-0.26297700405121,51.52098846435550
9,610244,PI/000055476,A G Wines,Retailers - other,4613,Exempt,fhrs_exempt_en-GB,2016-02-12,509,Ealing,...,West Ealing,NaN,NaN,W13 8QA,NaN,NaN,NaN,NaN,-0.31534900000000,51.51617000000000
